## Welcome to Week 4, Day 4

This is the start of an AWESOME project! Really simple and very effective.

In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain.agents import Tool

from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

In [3]:
load_dotenv(override=True)

True

### Asynchronous LangGraph

To run a tool:  
Sync: `tool.run(inputs)`  
Async: `await tool.arun(inputs)`

To invoke the graph:  
Sync: `graph.invoke(state)`  
Async: `await graph.ainvoke(state)`

In [4]:
# TypeDict is used instead of Pydantic object
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [18]:
# Here a tool is created using the Tool wrapper
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": text})

tool_push = Tool(
        name="send_push_notification",
        func=push,
        description="useful for when you want to send a push notification"
    )

### Next: Install Playwright

On Windows and MacOS:  
`playwright install`

On Linux:  
`playwright install —with-reps chromium`

Student Mariano R. mentioned that you might also need to do this in a cursor terminal:

`uv run playwright install chromium`

In [19]:
!playwright install

You are using a frozen webkit browser which does not receive updates anymore on mac13. Please update to the latest version of your operating system to test up-to-date browsers.


## And now - after Installing Playwright, a heads up for Windows PC Users:

While executing the next few cells, you might hit a problem with the Playwright browser raising a NotImplementedError.

This should work when we move to python modules, but it can cause problems in Windows in a notebook.

If you it this error and would like to run the notebook, you need to make a small change which seems quite hacky! You need to do this AFTER installing Playwright (prior cells)

1. Right click in `.venv` in the File Explorer on the left and select "Find in folder"
2. Search for `asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())`  
3. That code should be found in a line of code in a file called `kernelapp.py`
4. Comment out the entire else clause that this line is a part of - see the fragment below. Be sure to have the "pass" statement after the ImportError line.
5. Restart the kernel by pressing the "Restart" button above

```python
        if sys.platform.startswith("win") and sys.version_info >= (3, 8):
            import asyncio
 
            try:
                from asyncio import WindowsProactorEventLoopPolicy, WindowsSelectorEventLoopPolicy
            except ImportError:
                pass
                # not affected
           # else:
            #    if type(asyncio.get_event_loop_policy()) is WindowsProactorEventLoopPolicy:
                    # WindowsProactorEventLoopPolicy is not compatible with tornado 6
                    # fallback to the pre-3.8 default of Selector
                    # asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())
```

Thank you to student Nicolas for finding this, and to Kalyan, Yaki, Zibin and Bhaskar for confirming that this worked for them! And to Vladislav for the extra pointers.

As an alternative, you can just move to a Python module (which we do anyway in Day 5)

In [20]:
# Introducing nest_asyncio
# Python async code only allows for one "event loop" processing aynchronous events.
# The `nest_asyncio` library patches this, and is used for special situations, if you need to run a nested event loop.

# Running the notebook is an event loop itself. 
# We need this to allow running Playwright in the same time

import nest_asyncio
nest_asyncio.apply()

### The LangChain community

One of the remarkable things about LangChain is the rich community around it.

Check this out:


In [21]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

# If you get a NotImplementedError here or later, see the Heads Up at the top of the notebook

async_browser = create_async_playwright_browser(headless=False)  # headful mode
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

In [22]:
# Here we see the tools made available by the PlayWrightBrowserToolkit
# IMPORTAT: this is a very big help since we can equip our agent with all these tools
for tool in tools:
    print(f"{tool.name}={tool}")

click_element=async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/federico.tognetti/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>
navigate_browser=async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/federico.tognetti/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>
previous_webpage=async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/federico.tognetti/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>
extract_text=async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/federico.tognetti/Library/Caches/ms-playwright/chromium-1169/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=136.0.7103.25>
extract_hyperlinks=async_browser=<Browser type=<BrowserType name=chromium executable_path=/User

In [24]:
# Create a dictionary mapping tool names to tool objects for easy lookup
# This is only needed to extract tools by name in the following lines
tool_dict = {tool.name:tool for tool in tools}

# Get the navigation tool that allows browsing to URLs
navigate_tool = tool_dict.get("navigate_browser")
# Get the text extraction tool that can scrape text from the current page
extract_text_tool = tool_dict.get("extract_text")

# Use the navigation tool to browse to CNN's homepage
# The arun() method is used for async execution
await navigate_tool.arun({"url": "https://www.cnn.com/"})
# Extract all text content from the CNN page and store it in 'text' variable
text = await extract_text_tool.arun({})

In [25]:
# textwrap.fill() formats text by wrapping it to a specified width (default 70 characters)
# making it more readable when printed to the console
import textwrap
print(textwrap.fill(text))

Breaking News, Latest News and Videos | CNN CNN values your feedback
1. How relevant is this ad to you? 2. Did you encounter any technical
issues? Video player was slow to load content Video content never
loaded Ad froze or did not finish loading Video content did not start
after ad Audio on ad was too loud Other issues Ad never loaded Ad
prevented/slowed the page from loading Content moved around while ad
loaded Ad was repetitive to ads I've seen previously Other issues
Cancel Submit Thank You! Your effort and contribution in providing
this feedback is much
appreciated. Close Ad Feedback Close icon US World Politics Business
Health Entertainment Style Travel Sports Science Climate Weather
Ukraine-Russia War Israel-Hamas War Games More US World Politics
Business Health Entertainment Style Travel Sports Science Climate
Weather Ukraine-Russia War Israel-Hamas War Games Watch Listen Live TV
Subscribe Sign in My Account Settings Newsletters Topics you follow
Sign out Your CNN account Sign 

In [26]:
all_tools = tools + [tool_push]

In [27]:
# Here we create an agent with tools
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(all_tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


In [ ]:

graph_builder = StateGraph(State)                                           # Step 2: start the graph builder with the state defined in Step 1
graph_builder.add_node("chatbot", chatbot)                                  # Step 3: add a regular node
graph_builder.add_node("tools", ToolNode(tools=all_tools))                  # Step 3: add the tools node
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")   # Step 4: add conditional edges for tools node
graph_builder.add_edge("tools", "chatbot")                                  # Step 4: add a regular edge
graph_builder.add_edge(START, "chatbot")                                    # Step 4: add a regular edge

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)                          # Step 5: compile the graph (adding the memory defined above)
display(Image(graph.get_graph().draw_mermaid_png()))

In [30]:
# Configuration for the conversation thread
# Set a specific thread ID to track the conversation
config = {"configurable": {"thread_id": "10"}}

# Async function that handles the chat
# Parameters:
# - user_input: the message entered by the user
# - history: conversation history (not used in this case)
async def chat(user_input: str, history):
    # Invokes the graph asynchronously with the user's message. IMPORTANT: here we use async invoke
    # Formats the message in the required format (role + content)
    result = await graph.ainvoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    # Returns only the last message from the response
    return result["messages"][-1].content

# Creates and launches the chat interface using Gradio
# - Uses the chat function as backend
# - Sets interface type to "messages" for a chat visualization
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
